### ***Bootstrap parameter estimation for hawkes processes with exponential kernel***
In this notebook functions as an empirical test of the framework described in the paper "Bootstrap inference for Hawkes processes"
The functions below simulate hawkes processes with exponential kernel and true parameters. Hereafter a variety of bootstrap schemes are implemented for estimating coverage probabilities of these parameters. Respectively the fixed intensity bootstrap and recursive intensity bootstrap are implemented and hereafter nonparametric versions of these are implemented.

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import chi2, norm
import matplotlib.pyplot as plt
import time
from ipywidgets import interact, FloatSlider, IntSlider
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import chi2
np.set_printoptions(precision=6, suppress=True)

In [42]:
def simulate_hawkes(mu, alpha, beta, T, M, k):
    """
    Simulate a single realization of a Hawkes process with exponential kernel using Ogata's thinning algorithm.
    
    Parameters
    ----------
    mu : float
        Baseline intensity.
    alpha : float
        Jump size for each event.
    beta : float
        Decay rate of the exponential kernel.
    T : float
        End time for simulation (after burn-in).
    M : float
        Upper bound for the intensity (thinning envelope).
    k : float
        Length of the burn-in period. We simulate over [-k, T] and discard times <= 0.
    
    Returns
    -------
    events : list of float
        Event times in (0, T] (post burn-in).
    """
    all_events = []
    t = -k  # We define the burn-in period as [-k, 0] and start the simulation here
    # Ogata's thinning algorithm
    while True:
        # generate candidate time
        u = np.random.exponential(scale=1.0 / M)
        t += u
        if t > T:
            break

        # compute conditional intensity at time t
        intensity = mu + sum(alpha * np.exp(-beta * (t - ti)) for ti in all_events)

        # thinning step
        if np.random.uniform() <= intensity / M:
            all_events.append(t)
            
    # discard burn-in events
        events = [ti for ti in all_events if ti > 0]
    return events 

def intensity_on_grid(mu, alpha, beta, events, T, n_points=500):
    t_grid = np.linspace(0, T, n_points)
    lam = np.array([mu + sum(alpha * beta * np.exp(-beta * (t - ti))
                              for ti in events if ti < t)
                    for t in t_grid])
    return t_grid, lam

def plot_hawkes_interactive_scaled(mu=0.5, alpha=0.8, beta=1.5,
                                   T=100, M=1.0, k=10.0):
    events = simulate_hawkes(mu, alpha, beta, T, M, k)
    t_grid, lam = intensity_on_grid(mu, alpha, beta, events, T)

    fig, ax1 = plt.subplots(figsize=(8, 5))
    ax1.step(events, np.arange(1, len(events)+1),
             where='post', label='Cumulative Events', color = 'blue')
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Cumulative Events')
    ax1.grid(True, linestyle='--', alpha=0.5, color = 'black')
    ax1.set_xlim(0, T)
    ax1.set_ylim(0, len(events) + 1)

    ax2 = ax1.twinx()
    ax2.plot(t_grid, lam, '-', label='Intensity λ(t)', color='red', alpha=0.5)
    ax2.set_ylim(lam.min(), lam.max() * 1.1)
    ax2.set_ylabel('Intensity λ(t)')

    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2,
               loc='upper left', fontsize='small')

    plt.title(f'Hawkes Process (μ={mu}, α={alpha}, β={beta})')
    plt.show()

interact(
    plot_hawkes_interactive_scaled,
    mu=FloatSlider(min=0.0, max=1.0, step=0.1, value=0.5, description='μ'),
    alpha=FloatSlider(min=0.0, max=2.0, step=0.1, value=0.8, description='α'),
    beta=FloatSlider(min=0.1, max=5.0, step=0.1, value=1.5, description='β'),
    T=IntSlider(min=10, max=200, step=10, value=100, description='T'),
    M=FloatSlider(min=0.5, max=100.0, step=0.1, value=1.0, description='M'),
    k=FloatSlider(min=0.0, max=50.0, step=1.0, value=10.0, description='k'))

interactive(children=(FloatSlider(value=0.5, description='μ', max=1.0), FloatSlider(value=0.8, description='α'…

<function __main__.plot_hawkes_interactive_scaled(mu=0.5, alpha=0.8, beta=1.5, T=100, M=1.0, k=10.0)>

We now define the helper functions for the bootstrap schemes and the monte carlo simulation to generate confidence intervals.

In [127]:

def simulate_replicates(N, mu, alpha, beta, T, M, k, seed=None):
    """
    Generate N Monte Carlo replicates of a Hawkes process.
    
    Parameters
    ----------
    N : int
        Number of replicates.
    mu, alpha, beta, T, M, k : as in simulate_hawkes
    
    Returns
    -------
    replicates : list of lists
        A list with N elements, each being the list of post burn-in event times.
    """
    # Set random seed for reproducibility
    if seed is not None:
        np.random.seed(seed)
    # generate N replicates of the Hawkes process using the simulate_hawkes function
    replicates = []
    for _ in range(N):
        events = simulate_hawkes(mu, alpha, beta, T, M, k)
        replicates.append(events)
    return replicates

# We define re reparameterized log-likelihood function for the Hawkes process
def hawkes_loglik(params, events, T):
    """
    Negative log-likelihood of a Hawkes process with exponential kernel.
    
    params: array-like [mu, alpha, beta]
    events: list/array of event times in (0, T]
    T: float, observation window end
    """
    mu, eta, beta = params
    # enforce on parameters for stability
    if mu <= 0 or eta < 0 or eta >= 1 or beta <= 0:
        return np.inf
    
    n = len(events)
    log_term = 0.0
    for i, ti in enumerate(events):
        # intensity at ti
        decay_sum = 0.0
        for tj in events[:i]:
            decay_sum += eta * beta * np.exp(-beta * (ti - tj))
        lam = mu + decay_sum
        if lam <= 0:
            return np.inf
        log_term += np.log(lam)
    
    # compensator: mu*T + eta * sum_j (1 - exp(-beta*(T - t_j)))
    compensator = mu * T + eta * np.sum(1 - np.exp(-beta * (T - np.array(events))))
    # we return the negative log-likelihood for minimization later
    return -(log_term - compensator)

def ell_T(events, theta, T):
    """
    The true log-likelihood of the Hawkes data
    """
    return -hawkes_loglik(theta, events, T)

def fit_hawkes_MLE(events, T, init_params=None):
    """
    Fit Hawkes MLE via Limited-memory Broyden-Fletcher-Goldfarb-Shanno (L-BFGS-B) method, and returns MLE params.
    """
    if init_params is None:
        init_params = np.array([len(events)/T, 0.5, 1.0])  # mu, eta, beta
    bounds = [(1e-6, None), (0, 1-1e-6), (1e-6, None)]
    res = minimize(hawkes_loglik, init_params, args=(events, T),
                   method='L-BFGS-B', bounds=bounds)
    return res.x

def Lambda(t, events, params):
    """
    Integrated intensity for Hawkes with exp kernel up to time t.
    """
    mu, alpha, beta = params
    # baseline part
    val = mu * t
    # kernel part: for each event tj < t, add alpha*(1 - exp(-beta*(t - tj)))
    ev = np.array(events)
    mask = ev < t
    val += alpha * np.sum(1 - np.exp(-beta * (t - ev[mask])))
    return val


def Lambda_inv(s, events, params, T, tol=1e-5, max_iter=50):
    """
    Invert the integrated intensity function for Hawkes with exponential kernel using newton-raphson.
    
    Solves Lambda(t) - s = 0 with starting guess t0 = (s / Lambda(T)) * T.
    
    Parameters
    ----------
    s : float
        Target integrated intensity.
    events : list of floats
        Observed event times (for the fixed intensity).
    params : array-like [mu, alpha, beta]
        Hawkes parameters.
    T : float
        Upper bound for t.
    tol : float
        Convergence tolerance on |Lambda(t) - s|.
    max_iter : int
        Maximum number of Newton steps.
    
    Returns
    -------
    t : float
        Approximate solution of Λ(t) = s.
    """
    # Precompute the integrated intensity and check range
    s_max = Lambda(T, events, params)
    if not (0 <= s <= s_max):
        raise ValueError(f"s={s} out of [0, Λ(T)={s_max}]")

    # 1) Try Newton–Raphson
    try:
        # initial guess
        t = (s / s_max) * T
        mu, eta, beta = params
        for _ in range(max_iter):
            # compute Λ(t) and λ(t)
            ev = np.array(events)
            mask = ev < t
            # Λ(t)
            L_t = mu*t + eta * np.sum(1 - np.exp(-beta*(t - ev[mask])))
            # λ(t)
            lam_t = mu + eta*beta * np.sum(np.exp(-beta*(t - ev[mask])))
            # Newton step
            diff = L_t - s
            if abs(diff) < tol:
                return t
            t -= diff/lam_t
            t = max(min(t, T), 0.0)
        # In case newton fails to converge we try bisection
        raise RuntimeError("Newton failed")
    except Exception:
        # 2) Bisection method
        low, high = 0.0, T
        while high - low > tol:
            mid = 0.5*(low + high)
            if Lambda(mid, events, params) < s:
                low = mid
            else:
                high = mid
        return 0.5*(low + high)


#  Compute and rescale transformed waiting times v_i^c for non parametric bootstrap.
def compute_rescaled_residuals(events, T, theta):
    """
    Compute rescaled waiting times residuals.
    Returns v_c so that sum(v_c)=n (or n+1 if you include the final interval).
    """
    mu, eta, beta = theta

    # Since we use alpha in the integrated intensity, we need to compute it from eta and beta.
    alpha = eta * beta

    # 1) compute the s_i = Λ(t_i) − Λ(t_{i−1})
    s_vals = []
    prev = 0.0
    for ti in events:
        s_vals.append(Lambda(ti, events, (mu, alpha, beta))
                      - Lambda(prev, events, (mu, alpha, beta)))
        prev = ti

    # 3) normalize so that sum(v_c) = n
    s = np.array(s_vals)
    n = len(s)
    total = s.sum()            
    v_c = s * n / total
    return v_c


def compute_finite_diff_hessian(loglik_func, theta_hat, events, T, eps=1e-6):
    """
    Approximate the Hessian matrix of loglik_func at theta_hat via central finite differences.
    Here we differ from the function in the paper make computation more efficient. The result is numerically the same.
    """
    p = len(theta_hat)
    H = np.zeros((p, p))
    base = loglik_func(theta_hat, events, T)
    # step size: eps * (1 + |theta|)
    #h = eps * (1 + np.abs(theta_hat))
    h = np.cbrt(eps) * (1+np.maximum(np.abs(theta_hat), 1.0))
    for i in range(p):
        for j in range(i, p):
            theta_ip = theta_hat.copy()
            theta_im = theta_hat.copy()
            theta_jp = theta_hat.copy()
            theta_jm = theta_hat.copy()
            
            theta_ip[i] += h[i]
            theta_im[i] -= h[i]
            theta_jp[j] += h[j]
            theta_jm[j] -= h[j]
            
            f_pp = loglik_func(theta_ip, events, T)
            f_pm = loglik_func(np.array([theta_ip[k] if k!=j else theta_ip[k]-h[j] for k in range(p)]), events, T)
            f_mp = loglik_func(np.array([theta_im[k] if k!=j else theta_im[k]+h[j] for k in range(p)]), events, T)
            f_mm = loglik_func(theta_im, events, T)
            
            H_ij = (f_pp - f_pm - f_mp + f_mm) / (4 * h[i] * h[j])
            H[i, j] = H_ij
            H[j, i] = H_ij
    return H


def asymptotic_inference(events, T, theta0, alpha=0.05, eps=1e-4):
    """
    Asymptotic CIs and LRT, with alpha-CI and stationarity/Hessian flags.
    """
    # 1) MLE
    theta_hat = fit_hawkes_MLE(events, T)
    mu_h, eta_h, beta_h = theta_hat
    
    # 2) Hessian of neg log-lik
    H_nll = compute_finite_diff_hessian(hawkes_loglik, theta_hat, events, T, eps=eps)
    cov_hat = np.linalg.inv(H_nll)
    
    # 3) Marginal CIs for (mu, eta, beta)
    z = norm.ppf(1 - alpha/2)
    se = np.sqrt(np.diag(cov_hat))
    ci_lower = theta_hat - z*se
    ci_upper = theta_hat + z*se
    
    # 4) Delta-method CI for alpha = eta*beta
    grad_alpha = np.array([0.0, beta_h, eta_h])
    var_alpha = grad_alpha @ cov_hat @ grad_alpha
    alpha_hat = eta_h * beta_h
    ci_alpha = (alpha_hat - z*np.sqrt(var_alpha),
                alpha_hat + z*np.sqrt(var_alpha))
    
    # 5) LRT
    ll_hat = ell_T(events, theta_hat, T)
    ll_0   = ell_T(events, theta0, T)
    LR_obs = 2*(ll_hat - ll_0)
    pval = 1 - chi2.cdf(LR_obs, df=len(theta0))
    LR_reject = int(pval < alpha)
    
    # 6) stationarity and Hessian checks

    # -- stationarity
    stationarity_fail = (eta_h >= 1)

    # -- Hessian (observed information) definiteness
    #    1) symmetrize and regularize
    H_sym = 0.5*(H_nll + H_nll.T)
    H_reg = H_sym + 1e-4*np.eye(H_sym.shape[0])

    #    2) try eigenvalues first
    try:
        eigs = np.linalg.eigvalsh(H_reg)
        hessian_fail = np.any(eigs <= 0)
    except np.linalg.LinAlgError:
        #    3) fallback: Cholesky test
        try:
            np.linalg.cholesky(H_reg)
            hessian_fail = False
        except np.linalg.LinAlgError:
            hessian_fail = True
    
    return {
        'theta_hat': theta_hat,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'ci_alpha': ci_alpha,
        'coverage': ((theta0 >= ci_lower) & (theta0 <= ci_upper)).astype(int),
        'coverage_alpha': int(theta0[1]*theta0[2] >= ci_alpha[0] and theta0[1]*theta0[2] <= ci_alpha[1]),
        'LR_obs': LR_obs,
        'LR_reject': LR_reject,
        'stationarity_fail': stationarity_fail,
        'hessian_fail': hessian_fail
    }

## Helper functions
def _draw_bootstrap_sample(events, theta_hat, T, s_max, invert_lambda):
    """Generate one bootstrap sample of event times via given inversion."""
    cum, s_vals = 0.0, []
    while True:
        v = np.random.exponential() # sample from the exponential distribution with parameter 1
        if cum + v > s_max: # check if the next event exceeds the integrated intensity evaluated at T
            break
        cum += v; s_vals.append(cum) # store the jump times
    return [invert_lambda(s, events, theta_hat, T) for s in s_vals] # maps back to original event times

def _compute_studentized_t(boot_est, boot_hess, theta_hat, events, T):
    """
    Return (B x p) t-stats and original SEs, using the true 'events'
    to recompute the original Hessian.
    """
    p = len(theta_hat)
    B = boot_est.shape[0]
    # compute original Hessian from events
    H_orig = compute_finite_diff_hessian(hawkes_loglik, theta_hat, events, T, eps=1e-4)
    cov_orig = np.linalg.inv(H_orig)
    # guard against negative variances
    var_orig = np.diag(cov_orig) / T
    var_orig[var_orig <= 0] = np.nan
    se_orig = np.sqrt(var_orig)

    t_stats = np.full((B, p), np.nan)
    for b in range(B):
        cov_b = np.linalg.inv(boot_hess[b])
        var_b = np.diag(cov_b) / T
        var_b[var_b <= 0] = np.nan
        se_b = np.sqrt(var_b)
        valid = ~np.isnan(se_b)
        t_stats[b, valid] = (boot_est[b, valid] - theta_hat[valid]) / se_b[valid]
    return t_stats, se_orig

def _symmetrized_ci(theta_hat, t_stats, se_orig, alpha_ci):
    """Compute symmetrized-t CI for each parameter."""
    p = len(theta_hat)
    ci_low = np.zeros(p); ci_up = np.zeros(p)
    ## coumpute confidence intervals for each parameter with 1-alpha_ci confidence level
    for j in range(p):
        tj = t_stats[:, j]; tj = tj[np.isfinite(tj)]
        if len(tj) == 0:
            ci_low[j] = ci_up[j] = np.nan # if t stats are all NaN, set CI to NaN
        else: # compute the CI
            med = np.median(tj)
            pool = np.abs(np.concatenate([tj, 2*med - tj]))
            q = np.quantile(pool, 1 - alpha_ci)
            ci_low[j] = theta_hat[j] - q * se_orig[j]
            ci_up[j]  = theta_hat[j] + q * se_orig[j]
    return ci_low, ci_up

def _studentized_alpha_ci(theta_hat, boot_est, boot_hess, se_orig_alpha, alpha_ci, T):
    """Compute studentized symmetrized CI for alpha = eta*beta."""
    alpha_hat = theta_hat[1]*theta_hat[2]
    B = boot_est.shape[0]
    t_alpha = np.full(B, np.nan)
    # compute t-stats for alpha for each bootstrap sample
    for b in range(B):
        eta_b, beta_b = boot_est[b,1], boot_est[b,2]
        grad_b = np.array([0.0, beta_b, eta_b])
        Sigma_b = np.linalg.inv(boot_hess[b])
        var_b = grad_b @ Sigma_b @ grad_b
        se_b  = np.sqrt(var_b / T) if var_b > 0 else np.nan
        if se_b > 0:
            t_alpha[b] = (eta_b*beta_b - alpha_hat) / se_b
    t_alpha = t_alpha[np.isfinite(t_alpha)]
    if len(t_alpha)==0:
        return (np.nan, np.nan)
    # Center and symmetrize confidence intervals
    med = np.median(t_alpha)
    pool = np.abs(np.concatenate([t_alpha, 2*med - t_alpha]))
    q = np.quantile(pool, 1 - alpha_ci)
    return (alpha_hat - q * se_orig_alpha, alpha_hat + q * se_orig_alpha)


def fixed_intensity_bootstrap(events, T, B, true_params, alpha_ci=0.05, seed=None):
    """
    Fixed-Intensity Bootstrap (FIB) with symmetrized-t CI 
    and percentile fallback on Hessian failure.
    """
    if seed is not None:
        np.random.seed(seed)
    theta_hat = fit_hawkes_MLE(events, T)
    mu_h, eta_h, beta_h = theta_hat

    # 1) sanity checks
    stationarity_fail = (eta_h >= 1)
    #H_nll            = compute_finite_diff_hessian(hawkes_loglik, theta_hat, events, T)
    #hessian_fail     = np.any(np.linalg.eigvalsh(H_nll) <= 0)
    

    H_nll = compute_finite_diff_hessian(hawkes_loglik, theta_hat, events, T)
    # symmetrize + tiny ridge
    H_sym = 0.5*(H_nll + H_nll.T)
    H_reg = H_sym + 1e-6*np.eye(H_sym.shape[0])
    try:
        eigs = np.linalg.eigvalsh(H_reg)
        hessian_fail = np.any(eigs <= 0)
    except np.linalg.LinAlgError:
        # fallback to Cholesky
        try:
            np.linalg.cholesky(H_reg)
            hessian_fail = False
        except np.linalg.LinAlgError:
            hessian_fail = True


    # 2) observed LR
    ll_hat, ll_0 = ell_T(events, theta_hat, T), ell_T(events, true_params, T)
    LR_obs       = 2 * (ll_hat - ll_0)

    # 3) bootstrap draws
    s_max       = Lambda(T, events, theta_hat) # fixing the intensity
    boot_est    = []
    LR_star     = []
    boot_hess = []   
    for _ in range(B):
        t_star = _draw_bootstrap_sample(events, theta_hat, T, s_max, Lambda_inv)
        th_b    = fit_hawkes_MLE(t_star, T)
        boot_est.append(th_b) # store the bootstrap estimates
        LR_star.append(2*(ell_T(t_star, th_b, T)
                          - ell_T(t_star, theta_hat, T))) #store the bootstrap Likelihood ratio
        H_B = compute_finite_diff_hessian(hawkes_loglik, th_b, t_star, T)
        boot_hess.append(H_B) #store the bootstrap Hessian

    boot_est   = np.vstack(boot_est) #stack the bootstrap estimates
    alpha_boot = boot_est[:,1] * boot_est[:,2] #estimate alpha for each bootstrap sample

    # 4) build CIs in case of Hessian failure
    if hessian_fail:
        # -- percentile fallback --
        ci_low   = np.percentile(boot_est,   100*(alpha_ci/2),    axis=0)
        ci_up    = np.percentile(boot_est,   100*(1-alpha_ci/2),  axis=0)
        ci_alpha = np.percentile(alpha_boot, [100*(alpha_ci/2),
                                             100*(1-alpha_ci/2)])
        t_stats  = None

    else:
        # -- studentized‐t block --
        #cov_orig      = np.linalg.inv(H_nll) # compute the covariance matrix (asymptotic covariance)
        try:
            cov_orig = np.linalg.inv(H_reg)
        except np.linalg.LinAlgError:
            cov_orig = np.linalg.pinv(H_reg)



        with np.errstate(invalid="ignore"):
            se_orig   = np.sqrt(np.diag(cov_orig) / T) 
        #Compute the t-stats for each bootstrap sample and confidence interval
        t_stats, _    = _compute_studentized_t(boot_est, boot_hess,
                                               theta_hat, events, T) 
        ci_low, ci_up = _symmetrized_ci(theta_hat, t_stats, se_orig, alpha_ci)

        # α = η·β studentized‐t
        grad0       = np.array([0.0, beta_h, eta_h])
        var0_alpha  = max(grad0 @ cov_orig @ grad0, 0.0)
        se0_alpha   = np.sqrt(var0_alpha / T)
        ci_alpha    = _studentized_alpha_ci(theta_hat, boot_est, boot_hess,
                                            se0_alpha, alpha_ci, T)

    # 5) coverage flags & return
    #check if the true parameters are in the confidence intervals
    coverage       = ((true_params >= ci_low) & (true_params <= ci_up)).astype(int) 
    alpha0         = true_params[1] * true_params[2]
    coverage_alpha = int(ci_alpha[0] <= alpha0 <= ci_alpha[1])
    boot_lr_rej    = int(np.mean(np.array(LR_star) >= LR_obs) < alpha_ci) # Likelihood ratio test rejection

    return {
        'theta_hat':        theta_hat,
        'ci_lower':         ci_low,
        'ci_upper':         ci_up,
        'ci_alpha':         ci_alpha,
        't_stats':          t_stats,
        'coverage':         coverage,
        'coverage_alpha':   coverage_alpha,
        'boot_lr_reject':   boot_lr_rej,
        'stationarity_fail':stationarity_fail,
        'hessian_fail':     hessian_fail
    }


def recursive_intensity_bootstrap(events, T, B, true_params, alpha_ci=0.05, seed=None):
    """
    Recursive-Intensity Bootstrap (RIB) with symmetrized-t CI 
    and percentile fallback on Hessian failure.
    """
    ### Follows the same structure as fixed_intensity_bootstrap ###
    if seed is not None:
        np.random.seed(seed)

    # 1) Fit original MLE
    theta_hat = fit_hawkes_MLE(events, T)
    mu_h, eta_h, beta_h = theta_hat

    # 2) Sanity checks

    stationarity_fail = (eta_h >= 1)
    H_orig = compute_finite_diff_hessian(hawkes_loglik, theta_hat, events, T)

    # robust PD test
    H_sym = 0.5 * (H_orig + H_orig.T)
    H_reg = H_sym + 1e-8 * np.eye(H_sym.shape[0])
    try:
        eigs = np.linalg.eigvalsh(H_reg)
        hessian_fail = np.any(eigs <= 0)
    except np.linalg.LinAlgError:
        # fallback to Cholesky
        try:
            np.linalg.cholesky(H_reg)
            hessian_fail = False
        except np.linalg.LinAlgError:
            hessian_fail = True


    # 3) Observed LRT statistic
    ll_hat, ll_0 = ell_T(events, theta_hat, T), ell_T(events, true_params, T)
    LR_obs       = 2 * (ll_hat - ll_0)

    # 4) Generate B bootstrap samples
    s_max = mu_h * T
    boot_est = []
    LR_star  = []
    boot_hess = []
    for _ in range(B):
        cum, t_star = 0.0, []
        while True:
            v = np.random.exponential() # sample from the exponential distribution with parameter 1
            if cum + v > s_max:
                break
            cum += v
            t_star.append(Lambda_inv(cum, t_star, theta_hat, T)) # maps back to original event times using the bootstrap sample

        th_b = fit_hawkes_MLE(t_star, T)
        boot_est.append(th_b)
        LR_star.append(2*(ell_T(t_star, th_b, T)
                          - ell_T(t_star, theta_hat, T)))
        H_B = compute_finite_diff_hessian(hawkes_loglik, th_b, t_star, T)
        boot_hess.append(H_B) # store the bootstrap Hessian

    boot_est  = np.vstack(boot_est) # stack the bootstrap estimates
    alpha_boot = boot_est[:,1] * boot_est[:,2]

    # 5) CI construction with fallback
    if hessian_fail:
        # percentile CIs
        ci_low   = np.percentile(boot_est,     100*(alpha_ci/2), axis=0)
        ci_up    = np.percentile(boot_est,     100*(1-alpha_ci/2), axis=0)
        ci_alpha = np.percentile(alpha_boot,   [100*(alpha_ci/2),
                                                100*(1-alpha_ci/2)])
        t_stats  = None

    else:
        # studentized-t CIs
        #cov_orig      = np.linalg.inv(H_orig)
        try:
            cov_orig = np.linalg.inv(H_reg)
        except np.linalg.LinAlgError:
            cov_orig = np.linalg.pinv(H_reg)

        with np.errstate(invalid="ignore"):
            se_orig    = np.sqrt(np.diag(cov_orig) / T)
        t_stats, _    = _compute_studentized_t(boot_est, boot_hess,
                                               theta_hat, events, T)
        ci_low, ci_up = _symmetrized_ci(theta_hat, t_stats, se_orig, alpha_ci)

        # studentized-t for alpha = η·β
        grad0       = np.array([0.0, beta_h, eta_h])
        var0_alpha  = max(grad0 @ cov_orig @ grad0, 0.0)
        se0_alpha   = np.sqrt(var0_alpha / T)
        ci_alpha    = _studentized_alpha_ci(theta_hat, boot_est, boot_hess,
                                            se0_alpha, alpha_ci, T)

    # 6) Coverage & return
    coverage       = ((true_params >= ci_low) & (true_params <= ci_up)).astype(int)
    alpha0         = true_params[1] * true_params[2]
    coverage_alpha = int(ci_alpha[0] <= alpha0 <= ci_alpha[1])
    boot_lr_reject = int(np.mean(np.array(LR_star) >= LR_obs) < alpha_ci)

    return {
        'theta_hat':         theta_hat,
        'ci_lower':          ci_low,
        'ci_upper':          ci_up,
        'ci_alpha':          ci_alpha,
        't_stats':           t_stats,
        'coverage':          coverage,
        'coverage_alpha':    coverage_alpha,
        'boot_lr_reject':    boot_lr_reject,
        'stationarity_fail':  stationarity_fail,
        'hessian_fail':       hessian_fail
    }


def np_fixed_intensity_bootstrap(events, T, B, true_params, alpha_ci=0.05, seed=None):
    """
    Nonparametric Fixed-Intensity Bootstrap (NP-FIB) with
    symmetrized-t CIs and percentile fallback on Hessian failure.
    """
    ### Follows the same structure as previous functions with the difference being the rescaling in step 4 ###
    if seed is not None:
        np.random.seed(seed)

    # 1) Fit original MLE
    theta_hat = fit_hawkes_MLE(events, T)
    mu_h, eta_h, beta_h = theta_hat

    # 2) Stationarity & Hessian checks

    stationarity_fail = (eta_h >= 1)
    H_orig = compute_finite_diff_hessian(hawkes_loglik, theta_hat, events, T)

    # robust PD test
    H_sym = 0.5 * (H_orig + H_orig.T)
    H_reg = H_sym + 1e-8 * np.eye(H_sym.shape[0])
    try:
        eigs = np.linalg.eigvalsh(H_reg)
        hessian_fail = np.any(eigs <= 0)
    except np.linalg.LinAlgError:
        # fallback to Cholesky
        try:
            np.linalg.cholesky(H_reg)
            hessian_fail = False
        except np.linalg.LinAlgError:
            hessian_fail = True

    # 3) Observed LR statistic
    ll_hat, ll_0 = ell_T(events, theta_hat, T), ell_T(events, true_params, T)
    LR_obs       = 2 * (ll_hat - ll_0)

    # 4) Rescale residuals usinf the helper function
    v_c      = compute_rescaled_residuals(events, T, theta_hat)
    s_max    = Lambda(T, events, theta_hat) # fixing the intensity

    # 5) Draw B bootstrap samples
    boot_est = []
    LR_star  = []
    boot_hess = []
    for _ in range(B):
        v_star = np.random.choice(v_c, size=len(v_c), replace=True)
        s_cum  = np.cumsum(v_star)
        t_star = [Lambda_inv(s, events, theta_hat, T)
                  for s in s_cum if s <= s_max]

        th_b = fit_hawkes_MLE(t_star, T)
        boot_est.append(th_b)
        LR_star.append(2 * (ell_T(t_star, th_b, T)
                             - ell_T(t_star, theta_hat, T)))
        H_B = compute_finite_diff_hessian(hawkes_loglik, th_b, t_star, T)
        boot_hess.append(H_B)

    boot_est   = np.vstack(boot_est)
    alpha_boot = boot_est[:,1] * boot_est[:,2]

    # 6) Build CIs
    if hessian_fail:
        # ------ Percentile fallback ------
        ci_low   = np.percentile(boot_est,   100*(alpha_ci/2),    axis=0)
        ci_up    = np.percentile(boot_est,   100*(1-alpha_ci/2),  axis=0)
        ci_alpha = np.percentile(alpha_boot, [100*(alpha_ci/2),
                                              100*(1-alpha_ci/2)])
        t_stats  = None

    else:
        # ------ Studentized-t CI ------
        #cov_orig      = np.linalg.inv(H_orig)

        try:
            cov_orig = np.linalg.inv(H_reg)
        except np.linalg.LinAlgError:
            cov_orig = np.linalg.pinv(H_reg)


        with np.errstate(invalid="ignore"):
            se_orig    = np.sqrt(np.diag(cov_orig) / T)

        # need to recompute boot_hess to feed into _compute_studentized_t
        t_stats, _    = _compute_studentized_t(boot_est, boot_hess,
                                               theta_hat, events, T)
        ci_low, ci_up = _symmetrized_ci(theta_hat, t_stats, se_orig, alpha_ci)

        # studentized-t for alpha
        grad0       = np.array([0.0, beta_h, eta_h])
        var0_alpha  = max(grad0 @ cov_orig @ grad0, 0.0)
        se0_alpha   = np.sqrt(var0_alpha / T)
        ci_alpha    = _studentized_alpha_ci(theta_hat, boot_est, boot_hess,
                                            se0_alpha, alpha_ci, T)

    # 7) Coverage & return
    coverage       = ((true_params >= ci_low) & (true_params <= ci_up)).astype(int)
    alpha0         = true_params[1] * true_params[2]
    coverage_alpha = int(ci_alpha[0] <= alpha0 <= ci_alpha[1])
    boot_lr_reject = int(np.mean(np.array(LR_star) >= LR_obs) < alpha_ci)

    return {
        'theta_hat':         theta_hat,
        'ci_lower':          ci_low,
        'ci_upper':          ci_up,
        'ci_alpha':          ci_alpha,
        't_stats':           t_stats,
        'coverage':          coverage,
        'coverage_alpha':    coverage_alpha,
        'boot_lr_reject':    boot_lr_reject,
        'stationarity_fail':  stationarity_fail,
        'hessian_fail':       hessian_fail
    }


def np_recursive_intensity_bootstrap(events, T, B, true_params, alpha_ci=0.05, seed=None):
    """
    Nonparametric Recursive-Intensity Bootstrap (NP-RIB) with
    symmetrized-t CIs and percentile fallback on Hessian failure.
    """
    ### Follows the same structure as previous function without fixing the intensity in step 4 but computing it for each sample ###

    if seed is not None:
        np.random.seed(seed)

    # 1) Fit original MLE
    theta_hat = fit_hawkes_MLE(events, T)
    mu_h, eta_h, beta_h = theta_hat

    # 2) Stationarity & Hessian checks

    stationarity_fail = (eta_h >= 1)
    H_orig = compute_finite_diff_hessian(hawkes_loglik, theta_hat, events, T)

    # robust PD test
    H_sym = 0.5 * (H_orig + H_orig.T)
    H_reg = H_sym + 1e-8 * np.eye(H_sym.shape[0])
    try:
        eigs = np.linalg.eigvalsh(H_reg)
        hessian_fail = np.any(eigs <= 0)
    except np.linalg.LinAlgError:
        # fallback to Cholesky
        try:
            np.linalg.cholesky(H_reg)
            hessian_fail = False
        except np.linalg.LinAlgError:
            hessian_fail = True


    # 3) Observed LR statistic
    ll_hat, ll_0 = ell_T(events, theta_hat, T), ell_T(events, true_params, T)
    LR_obs       = 2 * (ll_hat - ll_0)

    # 4) Rescale residuals
    v_c      = compute_rescaled_residuals(events, T, theta_hat)
    s_max    = mu_h * T

    # 5) Draw B bootstrap samples
    boot_est    = []
    alpha_boot  = []
    LR_star     = []
    boot_hess   = []
    for _ in range(B):
        v_star = np.random.choice(v_c, size=len(v_c), replace=True)
        cum, t_star = 0.0, []
        for v in v_star:
            if cum + v > s_max:
                break
            cum += v
            t_star.append(Lambda_inv(cum, t_star, theta_hat, T))

        th_b = fit_hawkes_MLE(t_star, T)
        boot_est.append(th_b)
        alpha_boot.append(th_b[1] * th_b[2])
        LR_star.append(2 * (ell_T(t_star, th_b, T)
                             - ell_T(t_star, theta_hat, T)))
        H_B = compute_finite_diff_hessian(hawkes_loglik, th_b, t_star, T)
        boot_hess.append(H_B)

    boot_est    = np.vstack(boot_est)
    alpha_boot  = np.array(alpha_boot)

    # 6) CI construction
    if hessian_fail:
        # ------ Percentile fallback ------
        ci_low   = np.percentile(boot_est,   100*(alpha_ci/2),    axis=0)
        ci_up    = np.percentile(boot_est,   100*(1-alpha_ci/2),  axis=0)
        ci_alpha = np.percentile(alpha_boot, [100*(alpha_ci/2),
                                              100*(1-alpha_ci/2)])
        t_stats  = None

    else:
        # ------ Studentized-t CI ------
        #cov_orig      = np.linalg.inv(H_orig)

        try:
            cov_orig = np.linalg.inv(H_reg)
        except np.linalg.LinAlgError:
            cov_orig = np.linalg.pinv(H_reg)



        with np.errstate(invalid="ignore"):
            se_orig    = np.sqrt(np.diag(cov_orig) / T)

        t_stats, _    = _compute_studentized_t(boot_est, boot_hess,
                                               theta_hat, events, T)
        ci_low, ci_up = _symmetrized_ci(theta_hat, t_stats, se_orig, alpha_ci)

        # studentized-t for alpha 
        grad0       = np.array([0.0, beta_h, eta_h])
        var0_alpha  = max(grad0 @ cov_orig @ grad0, 0.0)
        se0_alpha   = np.sqrt(var0_alpha / T)
        ci_alpha    = _studentized_alpha_ci(theta_hat, boot_est, boot_hess,
                                            se0_alpha, alpha_ci, T)

    # 7) Coverage & return
    coverage       = ((true_params >= ci_low) & (true_params <= ci_up)).astype(int)
    alpha0         = true_params[1] * true_params[2]
    coverage_alpha = int(ci_alpha[0] <= alpha0 <= ci_alpha[1])
    boot_lr_reject = int(np.mean(np.array(LR_star) >= LR_obs) < alpha_ci)

    return {
        'theta_hat':         theta_hat,
        'ci_lower':          ci_low,
        'ci_upper':          ci_up,
        'ci_alpha':          ci_alpha,
        't_stats':           t_stats,
        'coverage':          coverage,
        'coverage_alpha':    coverage_alpha,
        'boot_lr_reject':    boot_lr_reject,
        'stationarity_fail':  stationarity_fail,
        'hessian_fail':       hessian_fail
    }

def monte_carlo_coverage_compare(
    mu, alpha, beta,
    T, M, k,
    B=50, R=20,
    alpha_ci=0.05,
    seed=None
):
    """
    Run R Monte Carlo replications, each with B bootstraps,
    and report estimated coverage for (μ,η,β,α) under:
      • Asymptotic
      • FIB, RIB, NP-FIB, NP-RIB
    """
    if seed is not None:
        np.random.seed(seed)

    theta0 = np.array([mu, alpha/beta, beta])

    # 1) Define a little adapter for the asymptotic method so it matches the bootstrap interface
    def asym_wrapper(events, T, B, theta0, alpha_ci, seed=None):
        out = asymptotic_inference(events, T, theta0, alpha=alpha_ci)
        return {
            'coverage':       out['coverage'],        # array [covμ,covη,covβ]
            'coverage_alpha': out['coverage_alpha'],  # 0/1
            'boot_lr_reject': out['LR_reject'],       # 0/1
        }

    # 2) putting all methods in a single dict
    methods = {
        'Asymptotic':            asym_wrapper,
        'FIB':                   fixed_intensity_bootstrap,
        'RIB':                   recursive_intensity_bootstrap,
        'NP-FIB':                np_fixed_intensity_bootstrap,
        'NP-RIB':                np_recursive_intensity_bootstrap,
    }

    # 3) Prepare counters
    hits    = {name: np.zeros(4, dtype=int) for name in methods}
    rejects = {name: 0 for name in methods}

    # 4) Monte Carlo loop
    for r in range(R):
        events = simulate_hawkes(mu, alpha, beta, T, M, k)
        for name, fn in methods.items():
            res = fn(events, T, B, theta0, alpha_ci, seed and seed + r)
            # accumulate coverage for μ,η,β
            hits[name][:3] += res['coverage']
            # accumulate coverage for α=ηβ
            hits[name][3]  += res['coverage_alpha']
            # accumulate LR rejection
            rejects[name]  += int(res['boot_lr_reject'])
            

    # 5) Build summary
    summary = {}
    for name in methods:
        summary[name] = {
            'cov(μ,η,β)': np.round(hits[name][:3] / R, 4),
            'cov(α)':      round(hits[name][3]    / R,    4),
            'rejLR':       round(rejects[name]    / R,    4),
        }
    return summary

def timed_monte_carlo(*args, **kwargs):
    t0 = time.perf_counter()
    out = monte_carlo_coverage_compare(*args, **kwargs)
    elapsed = time.perf_counter() - t0
    print(f"Total MC + bootstraps elapsed: {elapsed:.1f} s")
    return out

def monte_carlo_sanity_failures(
    configs,         # list of (mu, alpha, beta) triples
    T, M, k,         # Hawkes simulation params
    R=20,           # Monte Carlo reps per config
    seed=None
):
    """
    For each (mu,alpha,beta) in `configs`, estimate by simulation:
      • P(stationarity_fail) under the asymptotic fit
      • P(hessian_fail)      under the asymptotic fit

    Returns a dict mapping config->(p_stationarity_fail, p_hessian_fail).
    """
    if seed is not None:
        np.random.seed(seed)

    results = {}
    for (mu, alpha, beta) in configs:
        theta0 = np.array([mu, alpha / beta, beta])
        stat_count = 0
        hess_count = 0

        for r in range(R):
            # 1) simulate one path
            events = simulate_hawkes(mu, alpha, beta, T, M, k)

            # 2) fit + diag via your asymptotic_inference
            out = asymptotic_inference(events, T, theta0)

            # 3) tally failures
            stat_count += int(out['stationarity_fail'])
            hess_count += int(out['hessian_fail'])

        results[(mu, alpha, beta)] = (
            stat_count / R,
            hess_count / R
        )

    return results


def monte_carlo_sanity_failures(configs, T, M_func, k, R=100, seed=None):
    if seed is not None:
        np.random.seed(seed)
    results = {}
    for mu, alpha, beta in configs:
        theta0 = np.array([mu, alpha/beta, beta])
        stat_count = 0
        hess_count = 0
        for _ in range(R):
            events = simulate_hawkes(mu, alpha, beta, T, M_func(mu, alpha, beta), k)
            out = asymptotic_inference(events, T, theta0)
            stat_count += int(out['stationarity_fail'])
            hess_count += int(out['hessian_fail'])
        results[(mu, alpha, beta)] = (stat_count / R, hess_count / R)
    return results


def M_func(mu, alpha, beta):
    # We choose C=3 
    return mu+alpha**(1.2) * beta**(-0.1)
    


In [110]:
### Sanity check ###
# Set the parameter cofiguration for sanity checks
configs = [
    (0.2, 0.5, 1.0),
    (0.2, 0.5, 25.0),
    (0.2, 0.8, 1.0),
    (0.2, 0.8, 25.0),
    (0.8, 0.5, 1.0),
    (0.8, 0.5, 25.0),
    (0.8, 0.8, 1.0),
    (0.8, 0.8, 25.0),
]

T = 50 # simulation time can be adjusted
k = 100
R = 20
seed = 123


# Compute results
results = monte_carlo_sanity_failures(configs, T, M_func, k, R=R, seed=seed)

# Build DataFrame
df = pd.DataFrame([
    {'Config': f"μ={mu}, α={alpha}, β={beta}",
     'P(stat_fail)': p_stat,
     'P(hess_fail)': p_hess}
    for (mu, alpha, beta), (p_stat, p_hess) in results.items()
])

df = df.set_index('Config')

# Display styled
styled = df.style.background_gradient(cmap='coolwarm', vmin=0, vmax=1).format("{:.2f}")
display(styled)


,P(stat_fail),P(hess_fail)
Config,,
"μ=0.2, α=0.5, β=1.0",0.00,0.30
"μ=0.2, α=0.5, β=25.0",0.00,0.45
"μ=0.2, α=0.8, β=1.0",0.00,0.05
"μ=0.2, α=0.8, β=25.0",0.00,0.55
"μ=0.8, α=0.5, β=1.0",0.00,0.15
"μ=0.8, α=0.5, β=25.0",0.00,0.05
"μ=0.8, α=0.8, β=1.0",0.00,0.25
"μ=0.8, α=0.8, β=25.0",0.00,0.25


In [109]:

def compare_grid_over_T(
    mus, alphas, betas, Ts,
    M_func,        # a function M(mu,alpha,beta) to compute M if needed
    k,
    B=199, R=20,
    alpha_ci=0.05,
    seed=None
):
    """
    For each combination of mu in `mus`, alpha in `alphas`, beta in `betas`, and T in `Ts`,
    runs the MC compare and returns a dict:
      results[(mu,alpha,beta,T)] = summary_dict
    """
    all_results = {}
    for mu in mus:
        for alpha in alphas:
            for beta in betas:
                # compute M (thinning envelope) as a function of parameters
                M = M_func(mu, alpha, beta)
                for T in Ts:
                    theta0 = np.array([mu, alpha/beta, beta])
                    print(f"\n→ Running MC for μ={mu}, α={alpha}, β={beta}, T={T}")
                    t0 = time.perf_counter()
                    summary = monte_carlo_coverage_compare(
                        mu, alpha, beta,
                        T, M, k,
                        B=B, R=R,
                        alpha_ci=alpha_ci,
                        seed=seed
                    )
                    elapsed = time.perf_counter() - t0
                    print(f"  done in {elapsed:.1f}s")
                    all_results[(mu, alpha, beta, T)] = summary
    return all_results

# Example of a simple M_func:

# Define your grids
mus    = [0.2, 0.8]
alphas = [0.5, 0.8]
betas  = [1.0, 25]
Ts     = [50, 100]

# Run the sweep
results = compare_grid_over_T(
    mus, alphas, betas, Ts,
    M_func=M_func,
    k=100,
    B=199, R=20,
    alpha_ci=0.05,
    seed=42
)

# Print neatly
for (mu, alpha, beta, T), summary in results.items():
    print(f"\nConfig μ={mu}, α={alpha}, β={beta}, T={T}")
    for method, stats in summary.items():
        cov_mu, cov_eta, cov_beta = stats['cov(μ,η,β)']
        cov_alpha = stats['cov(α)']
        rejLR     = stats['rejLR']
        print(f"  {method:8s}: covμ={cov_mu:.2f}, covη={cov_eta:.2f}, covβ={cov_beta:.2f}, covα={cov_alpha:.2f}, rejLR={rejLR:.2f}")



→ Running MC for μ=0.2, α=0.5, β=1.0, T=50
  done in 88.8s

→ Running MC for μ=0.2, α=0.5, β=1.0, T=100
  done in 256.2s

→ Running MC for μ=0.2, α=0.5, β=25, T=50
  done in 45.5s

→ Running MC for μ=0.2, α=0.5, β=25, T=100
  done in 118.3s

→ Running MC for μ=0.2, α=0.8, β=1.0, T=50
  done in 180.5s

→ Running MC for μ=0.2, α=0.8, β=1.0, T=100
  done in 595.6s

→ Running MC for μ=0.2, α=0.8, β=25, T=50
  done in 46.9s

→ Running MC for μ=0.2, α=0.8, β=25, T=100
  done in 123.9s

→ Running MC for μ=0.8, α=0.5, β=1.0, T=50
  done in 818.7s

→ Running MC for μ=0.8, α=0.5, β=1.0, T=100
  done in 3473.9s

→ Running MC for μ=0.8, α=0.5, β=25, T=50
  done in 435.1s

→ Running MC for μ=0.8, α=0.5, β=25, T=100
  done in 1720.1s

→ Running MC for μ=0.8, α=0.8, β=1.0, T=50
  done in 1253.8s

→ Running MC for μ=0.8, α=0.8, β=1.0, T=100
  done in 5553.4s

→ Running MC for μ=0.8, α=0.8, β=25, T=50
  done in 433.8s

→ Running MC for μ=0.8, α=0.8, β=25, T=100
  done in 1697.3s

Config μ=0.2, α=0.5, 

In [ ]:
##Lengt of confidence intervals

#modifying the simulation function
def simulate_hawkes1(mu, alpha, beta, T, M=mu+alpha**(1.2)*beta**(-0.1), k=100.0):
    """
    Simulate Hawkes process via Ogata's thinning algorithm.
    """
    all_events = []
    t = -k
    while True:
        u = np.random.exponential(scale=1.0 / M)
        t += u
        if t > T:
            break
        intensity = mu + sum(alpha * np.exp(-beta * (t - ti)) for ti in all_events if t > ti)
        if np.random.uniform() <= intensity / M:
            all_events.append(t)
    
    events = [ti for ti in all_events if ti > 0]
    return events

# Event Generator Wrapper
def events_generator(config, T):
    mu, eta, beta = config
    alpha = eta * beta
    return simulate_hawkes1(mu, alpha, beta, T)

# CI Length Extraction 
def get_ci_lengths_all_methods(events, T, config, B=199, alpha=0.05):
    mu0, eta0, beta0 = config
    true_params = np.array(config)
    results = []

    methods = [
        ("Asymptotic", asymptotic_inference(events, T, true_params, alpha)),
        ("FIB", fixed_intensity_bootstrap(events, T, B, true_params, alpha)),
        ("RIB", recursive_intensity_bootstrap(events, T, B, true_params, alpha)),
        ("NP-FIB", np_fixed_intensity_bootstrap(events, T, B, true_params, alpha)),
        ("NP-RIB", np_recursive_intensity_bootstrap(events, T, B, true_params, alpha)),
    ]

    for method_name, result in methods:
        ci_low = np.array(result["ci_lower"], dtype=float)
        ci_up = np.array(result["ci_upper"], dtype=float)
        ci_len = ci_up - ci_low
        alpha_len = result["ci_alpha"][1] - result["ci_alpha"][0] if not any(np.isnan(result["ci_alpha"])) else np.nan

        results.append({
            "method": method_name,
            "mu": mu0,
            "eta": eta0,
            "beta": beta0,
            "length_mu": ci_len[0],
            "length_eta": ci_len[1],
            "length_beta": ci_len[2],
            "length_alpha": alpha_len,
            "stationarity_fail": result.get("stationarity_fail", None),
            "hessian_fail": result.get("hessian_fail", None)
        })

    return results

#Run All Configs
def run_all_configs(events_generator, T, configs, B=199, alpha=0.05):
    all_results = []
    for config in configs:
        events = events_generator(config, T)
        method_results = get_ci_lengths_all_methods(events, T, config, B, alpha)
        all_results.extend(method_results)
    
    return pd.DataFrame(all_results)

# Configs and Execution
configs = [
    (0.2, 0.5, 1.0),
    (0.2, 0.5, 25.0),
    (0.2, 0.8, 1.0),
    (0.2, 0.8, 25.0),
    (0.8, 0.5, 1.0),
    (0.8, 0.5, 25.0),
    (0.8, 0.8, 1.0),
    (0.8, 0.8, 25.0),
]

# Run
df = run_all_configs(events_generator, T=100, configs=configs, B=199)
print(df)